In [1]:
import requests
from pprint import pprint
from pymongo import MongoClient
from time import sleep
# from apscheduler.schedulers.background import BackgroundScheduler, BlockingScheduler
import yaml
# from database_connection import connect
from telethon.sync import TelegramClient#, events
import time
from mongodb_connection import mongodb_connect
import schedule
from querydb_actions import get_wallet_position
import os
from datetime import datetime

(db, pairs, user_positions, user_notifications, telegram_metadata, subscription) = mongodb_connect()

config_stream = open("../common_config.yml",'r')
config = yaml.load(config_stream, Loader=yaml.BaseLoader)

API_ID = config['telegram']['api_id']
API_HASH = config['telegram']['api_hash']
BOT_TOKEN = config['telegram']['bot_token']
session_name = "sessions/Bot"
URL = "https://api.telegram.org/bot{}/".format(BOT_TOKEN)

if not os.path.exists(os.getcwd()+os.sep+"sessions"):
    os.mkdir(os.getcwd()+os.sep+"sessions")
    with open(os.getcwd()+os.sep+"sessions"+os.sep+'Bot.session', 'w') as fp:
        pass
client = TelegramClient(session_name, API_ID, API_HASH).start(bot_token=BOT_TOKEN)

Initializing configurations...
Connecting to MongoDB Client...
Getting data collections...


# Broken down code

In [8]:
query = subscription.find()
users = query[0]
users

{'_id': ObjectId('64b63ced7f91294109306751'),
 'user_id': '6278581236',
 'wallets': ['0x7308a21030ae55721707fd4717bf5f8e1b0afbed',
  '0x3ec92c9d09403a76bda445ffdfaf6de59717219f',
  '0x0228028a0c92cfd9743e561a96b16edbb4606054']}

In [10]:
user_id = int(users["user_id"])
wallets = users["wallets"]
user_id, wallets

(6278581236,
 ['0x7308a21030ae55721707fd4717bf5f8e1b0afbed',
  '0x3ec92c9d09403a76bda445ffdfaf6de59717219f',
  '0x0228028a0c92cfd9743e561a96b16edbb4606054'])

In [ ]:
def get_wallet_position(collection_db, wallet_id: str):

    print("Trying to fetch wallet position...")
    
    res = collection_db.find_one({'wallet_id': wallet_id})
    if res:
        # print(f'\nResult: {res}\n')
        # doc_id = str(res[-1]['_id'])
        position_values = f'''Wallet Id:  {res["wallet_id"]},
        \nCollateral Symbol:  {res["collateral_symbol"]},
        \nCollateral Name:  {res["collateral_name"]},
        \nBorrowed Asset Share:  {res["borrowedAssetShare"]},
        \nDeposited Collateral Amount:  {res["depositedCollateralAmount"]},
        \nLent Asset Share:  {res["lentAssetShare"]}\n
        '''
    else:
        position_values = None
    return position_values

In [46]:
wallet_id = '0x0228028a0c92cfd9743e561a96b16edbb4606054'
collection_db = user_notifications

res = collection_db.find_one({'wallet_id': wallet_id})
pos = res['positions'][0]
res,pos

({'_id': ObjectId('64bd080457b14edcc9f9069f'),
  'wallet_id': '0x0228028a0c92cfd9743e561a96b16edbb4606054',
  'hlink_etherscanner': 'https://etherscan.io/address/0x0228028a0c92cfd9743e561a96b16edbb4606054',
  'hlink_fraxfacts': 'https://facts.frax.finance/fraxlend/users/0x0228028a0c92cfd9743e561a96b16edbb4606054',
  'positions': [{'pos_datetime': '2023-07-18 04:04:47',
    'pair_id': '0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15',
    'pair_symbol': 'crvFRAX/FRAX',
    'collateral_symbol': 'crvFRAX',
    'pair_ex_rate': 1967.83,
    'pair_borrow_APR': 3.0,
    'pair_lend_APR': 2.36,
    'user_borrow_amt_scaled': 2.37,
    'user_dep_col_amt_scaled': 0.96,
    'user_lent_amt_scaled': 0.0,
    'user_current_LTV': 0.0,
    'user_liquidation_price_scaled': 3.29}]},
 {'pos_datetime': '2023-07-18 04:04:47',
  'pair_id': '0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15',
  'pair_symbol': 'crvFRAX/FRAX',
  'collateral_symbol': 'crvFRAX',
  'pair_ex_rate': 1967.83,
  'pair_borrow_APR': 3.0,
  'pair_len

In [ ]:
pos = res['positions']
if len(pos)==0:
    print_msg = "This wallet has no valid positions"
    print(print_msg)
else:
    print_msg = f"This wallet has {len(pos)} valid position(s) - "
    print(print_msg)

In [35]:
# # Wallet Id:  {res["wallet_id"]},
# position_values = f'''\n#########################################
#         \nPair Id:  {pos["pair_id"]},
#         \nPair Symbol:  {pos["pair_symbol"]},
#         \nExchange Rate:  {pos["pair_ex_rate"]} FRAX,
#         \nBorrow APR:  {pos["pair_borrow_APR"]} %,
#         \nLend APR:  {pos["pair_lend_APR"]} %,
#         \nBorrwed Amount:  {pos["user_borrow_amt_scaled"]} FRAX,
#         \nDeposited Collateral:  {pos["user_dep_col_amt_scaled"]} {pos["collateral_symbol"]},
#         \nLent Amount:  {pos["user_lent_amt_scaled"]} FRAX,
#         \nCurrent LTV:  {pos["user_current_LTV"]} %,
#         \nLiquidation Price:  {pos["user_liquidation_price_scaled"]} FRAX,
#         '''
position_values = f'''\nPair Id:  {pos["pair_id"]},
Pair Symbol:  {pos["pair_symbol"]},
Exchange Rate:  {pos["pair_ex_rate"]} FRAX,
Borrow APR:  {pos["pair_borrow_APR"]} %,
Lend APR:  {pos["pair_lend_APR"]} %,
Borrwed Amount:  {pos["user_borrow_amt_scaled"]} FRAX,
Deposited Collateral:  {pos["user_dep_col_amt_scaled"]} {pos["collateral_symbol"]},
Lent Amount:  {pos["user_lent_amt_scaled"]} FRAX,
Current LTV:  {pos["user_current_LTV"]} %,
Liquidation Price:  {pos["user_liquidation_price_scaled"]} FRAX,
'''

In [36]:
print(position_values+position_values)


Pair Id:  0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15,
Pair Symbol:  crvFRAX/FRAX,
Exchange Rate:  1969.06 FRAX,
Borrow APR:  3.0 %,
Lend APR:  2.36 %,
Borrwed Amount:  2.37 FRAX,
Deposited Collateral:  0.96 crvFRAX,
Lent Amount:  0.0 FRAX,
Current LTV:  0.0 %,
Liquidation Price:  3.29 FRAX,

Pair Id:  0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15,
Pair Symbol:  crvFRAX/FRAX,
Exchange Rate:  1969.06 FRAX,
Borrow APR:  3.0 %,
Lend APR:  2.36 %,
Borrwed Amount:  2.37 FRAX,
Deposited Collateral:  0.96 crvFRAX,
Lent Amount:  0.0 FRAX,
Current LTV:  0.0 %,
Liquidation Price:  3.29 FRAX,



In [11]:

msg = get_wallet_position(user_notifications, wallet_id)
msg

Trying to fetch wallet position...

Result: {'_id': ObjectId('64bcedd36baa114d36aa3721'), 'wallet_id': '0x0228028a0c92cfd9743e561a96b16edbb4606054', 'positions': [{'pos_datetime': '2023-07-18 04:04:47', 'pair_id': '0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15', 'pair_symbol': 'crvFRAX/FRAX', 'collateral_symbol': 'crvFRAX', 'pair_ex_rate': 1969.06, 'pair_borrow_APR': 3.0, 'pair_lend_APR': 2.36, 'user_borrow_amt_scaled': 2.37, 'user_dep_col_amt_scaled': 0.96, 'user_lent_amt_scaled': 0.0, 'user_current_LTV': 0.0, 'user_liquidation_price_scaled': 3.29}]}



KeyError: 'collateral_symbol'

In [ ]:
# for wallet_id in wallets:
wallet_id = '0x0228028a0c92cfd9743e561a96b16edbb4606054'
try:
    client.send_message(user_id, 'Daily notifications for your wallet(s) are here!')
    msg = get_wallet_position(wallet, wallet_id) + "\n" + "Sent at: "+ date_time
    client.send_message(user_id,msg)
except:
    print("Error in sending notification... ")


In [ ]:






# Schedule the notification to be sent every day at a specific time
# s = schedule.every(60*30).seconds.do(send_notification)
s = schedule.every(20).seconds.do(send_notification)

# s = schedule.every().day.at("08:00:00", "America/New_York").do(send_notification)
print("\n",s.next_run)

# Start an infinite loop to run the scheduler
while True:
    schedule.run_pending()
    time.sleep(2)
    # time.sleep(200)

# Modularised code

In [3]:
def get_wallet_position(collection_db, wallet_id: str):

    print("Trying to fetch wallet position...")
    # get information for each wallet id
    res = collection_db.find_one({'wallet_id': wallet_id})
    if res:
        if res['positions']:
            # position_values = """"""
            str_position_values = f"This wallet- {wallet_id} has {len(res['positions'])} valid position(s) - "
            for pos in res['positions']:
                str_position_values += f'''\n
Pair Id:  {pos["pair_id"]},
Pair Symbol:  {pos["pair_symbol"]},
Exchange Rate:  {pos["pair_ex_rate"]} FRAX,
Borrow APR:  {pos["pair_borrow_APR"]} %,
Lend APR:  {pos["pair_lend_APR"]} %,
Borrwed Amount:  {pos["user_borrow_amt_scaled"]} FRAX,
Deposited Collateral:  {pos["user_dep_col_amt_scaled"]} {pos["collateral_symbol"]},
Lent Amount:  {pos["user_lent_amt_scaled"]} FRAX,
Current LTV:  {pos["user_current_LTV"]} %,
Liquidation Price:  {pos["user_liquidation_price_scaled"]} FRAX,
Last Updated on: {pos["pos_datetime"]}
    '''
        else:
            str_position_values = f"This wallet - {wallet_id} has no valid positions.\n"

        str_hyper_links = f"""\nFor more information, visit-
etherscan.io - {res['hlink_etherscanner']}
facts.frax.finance - {res['hlink_fraxfacts']}
"""
        str_position_values += str_hyper_links


    else:
        str_position_values = None
    return str_position_values



In [58]:
wallet_id = '0x0228028a0c92cfd9743e561a96b16edbb4606054'

msg_user_notif = get_wallet_position(user_notifications, wallet_id)
print(msg_user_notif)

Trying to fetch wallet position...
This wallet- 0x0228028a0c92cfd9743e561a96b16edbb4606054 has 1 valid position(s) - 

Pair Id:  0x78bb3aec3d855431bd9289fd98da13f9ebb7ef15,
Pair Symbol:  crvFRAX/FRAX,
Exchange Rate:  1967.83 FRAX,
Borrow APR:  3.0 %,
Lend APR:  2.36 %,
Borrwed Amount:  2.37 FRAX,
Deposited Collateral:  0.96 crvFRAX,
Lent Amount:  0.0 FRAX,
Current LTV:  0.0 %,
Liquidation Price:  3.29 FRAX,
Last Updated on: 2023-07-18 04:04:47
    
For more information, visit-
etherscan.io - https://etherscan.io/address/0x0228028a0c92cfd9743e561a96b16edbb4606054
facts.frax.finance - https://facts.frax.finance/fraxlend/users/0x0228028a0c92cfd9743e561a96b16edbb4606054



In [9]:
def send_notification():
    # with client :
        # Send a message to your bot
        query = subscription.find()
        # iterate over wallet_ids
        for users in query:
            user_id = int(users["user_id"])
            wallets = users["wallets"]
            for wallet_id in wallets:
                try:
                    # client.send_message(user_id, 'Daily notifications for your wallet(s) are here!')
                    msg_user_notif = get_wallet_position(user_notifications, wallet_id)
                    print(msg_user_notif)
                    # client.send_message(user_id,msg_user_notif)
                except:
                    print("Error in sending notification... ")


In [10]:
send_notification()

Trying to fetch wallet position...
This wallet- 0x7308a21030ae55721707fd4717bf5f8e1b0afbed has 1 valid position(s) - 

Pair Id:  0x794f6b13fbd7eb7ef10d1ed205c9a416910207ff,
Pair Symbol:  crvFRAX/FRAX,
Exchange Rate:  1875.26 FRAX,
Borrow APR:  0.0 %,
Lend APR:  0.0 %,
Borrwed Amount:  500581.3 FRAX,
Deposited Collateral:  650.0 crvFRAX,
Lent Amount:  0.0 FRAX,
Current LTV:  9.08 %,
Liquidation Price:  1026.83 FRAX,
Last Updated on: 2023-02-10 17:57:11
    
For more information, visit-
etherscan.io - https://etherscan.io/address/0x7308a21030ae55721707fd4717bf5f8e1b0afbed
facts.frax.finance - https://facts.frax.finance/fraxlend/users/0x7308a21030ae55721707fd4717bf5f8e1b0afbed

Trying to fetch wallet position...
This wallet- 0x3ec92c9d09403a76bda445ffdfaf6de59717219f has 1 valid position(s) - 

Pair Id:  0x50e627a1df8d665524942ad7ec6392b6ba60293a,
Pair Symbol:  crvFRAX/FRAX,
Exchange Rate:  1.08 FRAX,
Borrow APR:  0.0 %,
Lend APR:  0.0 %,
Borrwed Amount:  14.89 FRAX,
Deposited Collateral:

In [61]:
# Schedule the notification to be sent every day at a specific time
# s = schedule.every(60*30).seconds.do(send_notification)
s = schedule.every(20).seconds.do(send_notification)

# s = schedule.every().day.at("08:00:00", "America/New_York").do(send_notification)
print("\n",s.next_run)

# Start an infinite loop to run the scheduler
while True:
    schedule.run_pending()
    time.sleep(2)
    # time.sleep(200)


 2023-07-23 17:13:47.579469


AttributeError: __enter__